In [0]:
import os
def load_data(path):
   input_file = os.path.join(path)
   with open(input_file, "r", encoding='utf-8') as f:
       data = f.read()

   return data.split('\n')

In [2]:
import collections

import helper
import numpy as np
# import project_tests as tests

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [0]:
# Add-on : Check framework versions being used 
import tensorflow as tf
import keras as K
import sys

In [0]:
# Load English data
english_sentences = load_data('small_vocab_en')
# Load French data
french_sentences = load_data('small_vocab_fr')

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [7]:
english_words_counter['the']

67628

In [8]:
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# tests.test_tokenize(tokenize)


text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short quick sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)

print()

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'quick': 1, 'the': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 1, 4, 5, 6, 7, 2, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 1, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short quick sentence .
  Output: [18, 19, 3, 20, 1, 21]


In [9]:
def pad(x, length=None):
    
    if length is None:
        length = max([len(seq) for seq in x])
    
    return pad_sequences(sequences=x, maxlen=length, padding='post')

# tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [2 1 4 5 6 7 2 8 9]
  Output: [2 1 4 5 6 7 2 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  1 13 14 15 16  3 17]
  Output: [10 11 12  1 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20  1 21]
  Output: [18 19  3 20  1 21  0  0  0  0]


In [29]:
def preprocess(x, y):
   
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(preprocess_y.shape[0],preprocess_y.shape[1], 1)
    print(preprocess_x.shape)
    print(preprocess_y.shape)
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

(137861, 15)
(137861, 21, 1)
Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [0]:
# tokenizer.word_index

## Models

- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

In [0]:
def logits_to_text(logits, tokenizer):
    
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])


In [0]:
from keras.callbacks import TensorBoard
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=1, write_graph=True)

# Model-1-RNN

In [31]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    input_seq = Input(shape=input_shape[1:])
    rnn = GRU(units=english_vocab_size, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn) 
                             
    model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 9s 84us/step - loss: 2.7130 - accuracy: 0.4615 - val_loss: 2.0535 - val_accuracy: 0.5357
Epoch 2/5
110288/110288 [==============================] - 9s 81us/step - loss: 1.7775 - accuracy: 0.5697 - val_loss: 1.6026 - val_accuracy: 0.5929
Epoch 3/5
110288/110288 [==============================] - 9s 81us/step - loss: 1.5204 - accuracy: 0.5980 - val_loss: 1.4494 - val_accuracy: 0.6068
Epoch 4/5
110288/110288 [==============================] - 9s 84us/step - loss: 1.3963 - accuracy: 0.6150 - val_loss: 1.3452 - val_accuracy: 0.6205
Epoch 5/5
110288/110288 [==============================] - 9s 82us/step - loss: 1.3038 - accuracy: 0.6331 - val_loss: 1.2625 - val_accuracy: 0.6435
new jersey est parfois chaud en l' et il il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [45]:
simple_rnn_model.predict(tmp_x[:1]).shape

(1, 21, 345)

In [21]:
simple_rnn_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 200)           121200    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           69345     
_________________________________________________________________
activation_2 (Activation)    (None, 21, 345)           0         
Total params: 190,545
Trainable params: 190,545
Non-trainable params: 0
_________________________________________________________________


# Model-2-Embedding

In [47]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    embedding_size = 128
    rnn_cells = 200
    dropout = 0.0
    learning_rate = 1e-3
   
    input_seq = Input(shape=input_shape[1:])
     
    embedded_seq = Embedding(input_dim = english_vocab_size, 
                             output_dim = embedding_size,
                             input_length=input_shape[1:][0])(input_seq)
    
    rnn = GRU(units=rnn_cells, dropout=dropout, return_sequences=True)(embedded_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn) 
    model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model    
    
# tests.test_embed_model(embed_model)


tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

embed_rnn_model = embed_model(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)


embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Tensor("gru_10/transpose_1:0", shape=(None, 21, 200), dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 10s 95us/step - loss: 3.2288 - accuracy: 0.4162 - val_loss: 2.5406 - val_accuracy: 0.4787
Epoch 2/5
110288/110288 [==============================] - 10s 89us/step - loss: 2.0423 - accuracy: 0.5524 - val_loss: 1.5588 - val_accuracy: 0.6311
Epoch 3/5
110288/110288 [==============================] - 10s 91us/step - loss: 1.3156 - accuracy: 0.6799 - val_loss: 1.1096 - val_accuracy: 0.7337
Epoch 4/5
110288/110288 [==============================] - 10s 91us/step - loss: 0.9561 - accuracy: 0.7630 - val_loss: 0.8190 - val_accuracy: 0.7867
Epoch 5/5
110288/110288 [==============================] - 10s 89us/step - loss: 0.7234 - accuracy: 0.8052 - val_loss: 0.6392 - val_accuracy: 0.8214
new jersey est parfois calme en l'automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# Model-3-BiDirectional RNN

In [19]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    dropout = 0.0
    learning_rate = 1e-3
    
    # Choose Sequential or Functional API implementation ('seq' or 'func')
    impl='seq'   
    if impl=='func':
        # Sequential Model 
        print("Using Sequential model (Note: this version makes the unitary test to fail: Disable tests to use it)")
        from keras.models import Sequential
        model = Sequential()
        model.add(Bidirectional(GRU(english_vocab_size, dropout=dropout, return_sequences=True)))
        model.add(Dense(french_vocab_size, activation='softmax'))
        
    else:
        # model's Functional equivalent
        # Note : we could have also used "Bidirectional(GRU(...))" instead of buidling the Bidirectional RNNS manually
        print("Using Functional API")
        from keras.layers import concatenate, add
        input_seq = Input(shape=input_shape[1:])
        right_rnn = GRU(units=english_vocab_size, return_sequences=True, go_backwards=False)(input_seq)
        left_rnn = GRU(units=english_vocab_size, return_sequences=True, go_backwards=True)(input_seq)

        # Choose how to merge the 2 rnn layers : add or concatenate
        #logits = TimeDistributed(Dense(units=french_vocab_size))(add([right_rnn, left_rnn])) 
        logits = TimeDistributed(Dense(units=french_vocab_size))(concatenate([right_rnn, left_rnn])) 
        
        model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
  
    return model
   


tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1)) # convert it to 3 dimention

bd_rnn_model = bd_model(input_shape = tmp_x.shape,
                           output_sequence_length = max_french_sequence_length,
                           english_vocab_size = english_vocab_size+1,
                           french_vocab_size = french_vocab_size+1)


bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Using Functional API
Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 16s 147us/step - loss: 2.5390 - accuracy: 0.4824 - val_loss: 1.8784 - val_accuracy: 0.5624
Epoch 2/5
110288/110288 [==============================] - 15s 139us/step - loss: 1.6599 - accuracy: 0.5853 - val_loss: 1.5188 - val_accuracy: 0.6022
Epoch 3/5
110288/110288 [==============================] - 15s 139us/step - loss: 1.4386 - accuracy: 0.6128 - val_loss: 1.3717 - val_accuracy: 0.6234
Epoch 4/5
110288/110288 [==============================] - 15s 138us/step - loss: 1.3166 - accuracy: 0.6332 - val_loss: 1.2668 - val_accuracy: 0.6372
Epoch 5/5
110288/110288 [==============================] - 15s 136us/step - loss: 1.2300 - accuracy: 0.6468 - val_loss: 1.1948 - val_accuracy: 0.6515
paris jersey est parfois calme en l' et il est il est en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [20]:
bd_rnn_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 21, 1)        0                                            
__________________________________________________________________________________________________
gru_4 (GRU)                     (None, 21, 200)      121200      input_4[0][0]                    
__________________________________________________________________________________________________
gru_5 (GRU)                     (None, 21, 200)      121200      input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 21, 400)      0           gru_4[0][0]                      
                                                                 gru_5[0][0]                

# Model 4: Encoder-Decoder 

In [22]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    embedding_size = 128
    rnn_cells = 200
    dropout = 0.0
    learning_rate = 1e-3
    
    from keras.layers import LSTM
    
    encoder_input_seq = Input(shape=input_shape[1:], name="enc_input")
 
    # Encoder (Return the internal states of the RNN -> 1 hidden state for GRU cells, 2 hidden states for LSTM cells))
    encoder_output, state_t = GRU(units=rnn_cells, 
                                  dropout=dropout,
                                  return_sequences=False,
                                  return_state=True,
                                  name="enc_rnn")(encoder_input_seq)
    #for LSTM cells: encoder_output, state_h, state_c = LSTM(...)
        
    # Decoder Input   
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)

    # Decoder RNN (Take the encoder returned states as initial states)
    decoder_out = GRU(units=rnn_cells,
                      dropout=dropout,
                      return_sequences=True,
                      return_state=False)(decoder_input_seq, initial_state=state_t)
    #for LSTM cells: (decoder_input_seq, initial_state=[state_h, state_c])
    
    # Decoder output 
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_out) 
    
    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
     
    return model    

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

encdec_rnn_model = encdec_model(input_shape = tmp_x.shape,
                                output_sequence_length = max_french_sequence_length,
                                english_vocab_size = english_vocab_size+1,
                                french_vocab_size = french_vocab_size+1)
    

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2) # callbacks=[tensorboard]

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
    
 

Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 17s 155us/step - loss: 2.8501 - accuracy: 0.4584 - val_loss: 2.2078 - val_accuracy: 0.5154
Epoch 2/5
110288/110288 [==============================] - 16s 148us/step - loss: 2.0137 - accuracy: 0.5365 - val_loss: 1.8046 - val_accuracy: 0.5571
Epoch 3/5
110288/110288 [==============================] - 16s 148us/step - loss: 1.6673 - accuracy: 0.5688 - val_loss: 1.6150 - val_accuracy: 0.5669
Epoch 4/5
110288/110288 [==============================] - 16s 146us/step - loss: 1.5485 - accuracy: 0.5829 - val_loss: 1.4995 - val_accuracy: 0.5926
Epoch 5/5
110288/110288 [==============================] - 16s 146us/step - loss: 1.4644 - accuracy: 0.6001 - val_loss: 1.4253 - val_accuracy: 0.6094
new jersey est parfois en en mois et il est est en en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# Model 5: Custom 

In [48]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    embedding_size = 128
    rnn_cells = 300
    dropout = 0.2
    learning_rate = 1e-3
    
    from keras.layers import LSTM, concatenate
    
    encoder_input_seq = Input(shape=input_shape[1:]) 
    embedded_input_seq = Embedding(input_dim = english_vocab_size,
                                   output_dim = embedding_size,
                                   input_length=input_shape[1:][0])(encoder_input_seq)
    
    ## Note Bidirectional LSTM is used on the encoder side (see https://arxiv.org/pdf/1609.08144.pdf )
    ## Alternate version : Encoder RNN Bidirectional layer (Using the Keras Bidirectional layer wrappers)
    #encoder_output, forward_state_h, forward_state_c, backward_state_h, backward_state_c = Bidirectional(LSTM(units=rnn_cells,
    #                                                                                                          dropout=dropout,
    #                                                                                                          return_sequences=False,
    #                                                                                                          return_state=True))(embedded_input_seq)
    
    # Encoder Forward RNN layer
    encoder_forward_output, forward_state_h, forward_state_c = LSTM(units=rnn_cells,
                                                                    dropout=dropout,
                                                                    return_sequences=False,
                                                                    return_state=True,
                                                                    go_backwards=False)(embedded_input_seq)
    
    # Encoder backward RNN layer
    encoder_backward_output, backward_state_h, backward_state_c = LSTM(units=rnn_cells,
                                                                       dropout=dropout,
                                                                       return_sequences=False,
                                                                       return_state=True,
                                                                       go_backwards=True)(embedded_input_seq)
    
    # Encoder output and states : Merge the LSTM Forward and Backward ouputs (using 'concatenate' method) 
    state_h = concatenate([forward_state_h, backward_state_h]) 
    state_c = concatenate([forward_state_c, backward_state_c])    
    encoder_output = concatenate([encoder_forward_output, encoder_backward_output])        
         
    # Decoder Input   
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)
    
    # Decoder RNN layer
    # Note : we need twice more LSTM cells as we have concatenated backward and forward LSTM layers in encoder
    decoder_output = LSTM(units=rnn_cells*2,
                                  dropout=dropout,
                                  return_sequences=True,
                                  return_state=False,
                                  go_backwards=False)(decoder_input_seq, initial_state=[state_h, state_c])
    
    # Decoder output
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output) 
    
    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model    
 
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

final_rnn_model = model_final(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)

print(final_rnn_model.summary(line_length=125))


final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
    

Model: "model_8"
_____________________________________________________________________________________________________________________________
Layer (type)                             Output Shape               Param #        Connected to                              
input_9 (InputLayer)                     (None, 21)                 0                                                        
_____________________________________________________________________________________________________________________________
embedding_4 (Embedding)                  (None, 21, 128)            25600          input_9[0][0]                             
_____________________________________________________________________________________________________________________________
lstm_1 (LSTM)                            [(None, 300), (None, 300), 514800         embedding_4[0][0]                         
_____________________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 39s 355us/step - loss: 2.5213 - accuracy: 0.4861 - val_loss: 1.6784 - val_accuracy: 0.5771
Epoch 2/10
110288/110288 [==============================] - 38s 343us/step - loss: 1.5167 - accuracy: 0.5987 - val_loss: 1.3257 - val_accuracy: 0.6321
Epoch 3/10
110288/110288 [==============================] - 38s 347us/step - loss: 1.2263 - accuracy: 0.6620 - val_loss: 1.0831 - val_accuracy: 0.6969
Epoch 4/10
110288/110288 [==============================] - 38s 347us/step - loss: 1.0324 - accuracy: 0.7053 - val_loss: 0.9341 - val_accuracy: 0.7292
Epoch 5/10
110288/110288 [==============================] - 38s 349us/step - loss: 0.9120 - accuracy: 0.7308 - val_loss: 0.8321 - val_accuracy: 0.7493
Epoch 6/10
110288/110288 [==============================] - 38s 344us/step - loss: 0.8062 - accuracy: 0.7555 - val_loss: 0.7764 - val_accuracy: 0.7625
Epoch 7/10
110288/110288 [=================

## Prediction

In [0]:
def final_predictions(x, y, x_tk, y_tk):
   
    model = model_final(input_shape = x.shape,
                        output_sequence_length = y.shape[1],
                        english_vocab_size = len(x_tk.word_index)+1,
                        french_vocab_size = len(y_tk.word_index)+1)

    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)